In [662]:
%matplotlib inline
import pandas as pd
import numpy as np
tweets_raw = pd.read_csv("data/Sentiment.csv")

In [687]:
sentiments = tweets_raw.loc[:,['sentiment', 'text']]
sentiments_pos = sentiments[sentiments.sentiment == 'Positive']
sentiments_neg = sentiments[sentiments.sentiment == 'Negative']

import re
def word_feats(tweet_full):
    words = re.findall(r"[\w']+|[.,!?;]", tweet_full[1])
    words_filtered = [e.lower() for e in words if len(e) >= 3]
    return dict([(word, True) for word in words_filtered])

neg_features = [(word_feats(tweet_full), 'neg') for tweet_full in sentiments_neg.values]
pos_features = [(word_feats(tweet_full), 'pos') for tweet_full in sentiments_pos.values]
sentiments_test = neg_features[:10] + pos_features[:10]
sentiments_train = neg_features[400:2236] + pos_features[400:2236]
print (len(neg_features))
print (len(pos_features))

8493
2236


In [688]:
import nltk
from nltk import NaiveBayesClassifier
clf = NaiveBayesClassifier.train(sentiments_train)
print ('accuracy:', nltk.classify.util.accuracy(clf, sentiments_test))
clf.show_most_informative_features()

accuracy: 0.8
Most Informative Features
              gopdebates = True              pos : neg    =    337.0 : 1.0
            rwsurfergirl = True              pos : neg    =     68.6 : 1.0
                    band = True              pos : neg    =     48.3 : 1.0
          donniewahlberg = True              pos : neg    =     37.7 : 1.0
                   nails = True              pos : neg    =     33.0 : 1.0
                together = True              pos : neg    =     29.8 : 1.0
                 ratings = True              pos : neg    =     24.4 : 1.0
                 tedcruz = True              pos : neg    =     22.1 : 1.0
                  headed = True              pos : neg    =     21.0 : 1.0
                 brought = True              pos : neg    =     19.7 : 1.0


In [696]:
import collections
reference_sets = collections.defaultdict(set)
test_sets = collections.defaultdict(set)
for i, features in enumerate(sentiments_test):
    label = features[1]
    reference_sets[label].add(i)
    predicted = clf.classify(features[0])
    test_sets[predicted].add(i)
    print (label + " vs " + predicted)
test_sets

neg vs neg
neg vs neg
neg vs neg
neg vs neg
neg vs neg
neg vs neg
neg vs neg
neg vs neg
neg vs neg
neg vs neg
pos vs neg
pos vs neg
pos vs pos
pos vs pos
pos vs neg
pos vs pos
pos vs pos
pos vs pos
pos vs neg
pos vs pos


defaultdict(set,
            {'neg': {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 14, 18},
             'pos': {12, 13, 15, 16, 17, 19}})

In [690]:
from nltk.metrics.scores import precision
from nltk.metrics.scores import recall
pos_precision = nltk.precision(reference_sets['pos'], test_sets['pos'])
pos_recall = nltk.recall(reference_sets['pos'], test_sets['pos'])
neg_precision = nltk.precision(reference_sets['neg'], test_sets['neg'])
neg_recall = nltk.recall(reference_sets['neg'], test_sets['neg'])

In [691]:
print ("accuracy: " + str(accuracy))
print ("pos_precision: " + str(pos_precision))
print ("pos_recall: " + str(pos_recall))
print ("neg_precision: " + str(neg_precision))
print ("neg_recall: " + str(neg_recall))

accuracy: 0.52
pos_precision: 1.0
pos_recall: 0.6
neg_precision: 0.7142857142857143
neg_recall: 1.0


In [692]:
sentiments_test[0][1]

'neg'